# Infer-1-Setup : Introduction et Installation

**Serie** : Programmation Probabiliste avec Infer.NET (1/13)  
**Duree estimee** : 15 minutes  
**Prerequis** : Notions de base en C# et statistiques

---

## Objectifs

- Comprendre les bases de la programmation probabiliste
- Installer et configurer Infer.NET
- Creer votre premier modele probabiliste
- Maitriser les 3 etapes fondamentales : modele, moteur, inference

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| - | [Infer-2-Gaussian-Mixtures](Infer-2-Gaussian-Mixtures.ipynb) |

---

## 1. Introduction a la Programmation Probabiliste

### Le probleme de l'incertitude

Les ordinateurs sont rigoureusement logiques, mais le monde reel ne l'est pas. Considerez ces exemples :

- Un systeme de reconnaissance d'ecriture manuscrite : le gribouillis peut correspondre a "hill", "bull" ou "hello"
- Un diagnostic medical : les symptomes peuvent indiquer plusieurs maladies possibles
- Un systeme de recommandation : les preferences de l'utilisateur sont partiellement connues

Dans tous ces cas, nous devons raisonner avec de l'**incertitude**.

### Variables aleatoires

Les variables conventionnelles (`bool`, `int`, `double`) ont des valeurs bien definies. La programmation probabiliste introduit les **variables aleatoires** qui representent un ensemble de valeurs possibles, chacune associee a une probabilite.

```csharp
// Variable classique - valeur fixe
bool estFace = true;

// Variable aleatoire - distribution de probabilite
Variable<bool> estFace = Variable.Bernoulli(0.5);  // 50% de chance
```

### Les 3 piliers de la programmation probabiliste

1. **Modeles probabilistes** : Definissent comment les observations sont generees
2. **Inference bayesienne** : Raisonne de l'effet vers la cause
3. **Apprentissage** : Les parametres eux-memes sont des variables aleatoires

## 2. Presentation d'Infer.NET

### Qu'est-ce que Infer.NET ?

**Infer.NET** est un framework Microsoft pour l'inference bayesienne dans les modeles graphiques. Il fait partie de la bibliotheque **ML.NET**.

**Caracteristiques principales** :

| Caracteristique | Description |
|-----------------|-------------|
| **Langage de modelisation** | Variables continues et discretes, univariees et multivariees |
| **Algorithmes d'inference** | Expectation Propagation (EP), Variational Message Passing (VMP), Gibbs Sampling |
| **Performance** | Compile les modeles en code source optimise |
| **Extensibilite** | Ajout de distributions, facteurs et algorithmes personnalises |

### Comment fonctionne Infer.NET ?

```
1. Definition du modele    -->  API de modelisation
        |
        v
2. Compilation du modele   -->  Generation de code source
        |
        v
3. Execution de l'inference -->  Calcul des distributions posterieures
```

## 3. Installation

Avant de commencer, nous devons installer les packages NuGet necessaires. Infer.NET est distribue en deux packages :
- **Microsoft.ML.Probabilistic** : Contient les distributions, le moteur d'inference et les algorithmes
- **Microsoft.ML.Probabilistic.Compiler** : Permet la compilation dynamique des modeles en code optimise

L'installation via `#r "nuget:` telecharge automatiquement les packages et leurs dependances.

> **Important** : Si Graphviz est installe, la cellule suivante l'ajoute automatiquement au PATH pour permettre la visualisation des graphes de facteurs.

In [1]:
// Configuration du PATH pour Graphviz (AVANT le chargement d'Infer.NET)
// Cette cellule doit s'executer en premier pour que Infer.NET trouve 'dot'

var graphvizPaths = new[] {
    @"C:\Program Files\Graphviz\bin",
    @"C:\Program Files (x86)\Graphviz\bin"
};

foreach (var gvPath in graphvizPaths)
{
    var dotExe = System.IO.Path.Combine(gvPath, "dot.exe");
    if (System.IO.File.Exists(dotExe))
    {
        var currentPath = Environment.GetEnvironmentVariable("PATH") ?? "";
        if (!currentPath.Contains(gvPath, StringComparison.OrdinalIgnoreCase))
        {
            Environment.SetEnvironmentVariable("PATH", gvPath + ";" + currentPath);
            Console.WriteLine($"Graphviz configure : {gvPath}");
        }
        else
        {
            Console.WriteLine($"Graphviz deja dans le PATH : {gvPath}");
        }
        break;
    }
}

// Verification avec ProcessStartInfo correctement configure
try
{
    var psi = new System.Diagnostics.ProcessStartInfo {
        FileName = "dot",
        Arguments = "-V",
        RedirectStandardError = true,
        RedirectStandardOutput = true,
        UseShellExecute = false,
        CreateNoWindow = true
    };
    using var proc = System.Diagnostics.Process.Start(psi);
    var stderr = proc.StandardError.ReadToEnd();
    proc.WaitForExit();
    if (proc.ExitCode == 0 || stderr.Contains("graphviz"))
        Console.WriteLine($"Verification OK : {stderr.Trim()}");
    else
        Console.WriteLine("dot.exe trouve mais verification echouee");
}
catch 
{ 
    Console.WriteLine("Graphviz non disponible (optionnel pour visualisation)"); 
}

The below script needs to be able to find the current output cell; this is an easy method to get it.

Graphviz configure : C:\Program Files\Graphviz\bin
Verification OK : dot - graphviz version 14.1.2 (20260124.0452)


### Verification de la configuration Graphviz

La cellule precedente a configure le PATH pour Graphviz. Voici comment interpreter la sortie :

| Message | Signification |
|---------|---------------|
| `Graphviz configure : C:\Program Files\Graphviz\bin` | Chemin ajoute au PATH - Graphviz sera utilisable |
| `Graphviz deja dans le PATH` | Deja configure dans une session precedente |
| `Graphviz non disponible` | Non installe - les graphes seront generes en `.gv` uniquement |
| `Verification OK : dot - graphviz version X.X` | Installation fonctionnelle |

> **Note** : Si Graphviz n'est pas installe, ce n'est pas bloquant. Les fichiers `.gv` peuvent etre visualises sur [viz-js.com](https://viz-js.com/).

In [2]:
// Installation des packages NuGet Infer.NET
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

> **Note** : Infer.NET fait desormais partie de la bibliotheque ML.NET de Microsoft.

In [3]:
// Import des espaces de noms essentiels
using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET charge avec succes !");
Console.WriteLine($"  - Microsoft.ML.Probabilistic.Models : Variables aleatoires et modeles");
Console.WriteLine($"  - Microsoft.ML.Probabilistic.Distributions : Gaussian, Beta, Dirichlet...");
Console.WriteLine($"  - Microsoft.ML.Probabilistic.Algorithms : EP, VMP, Gibbs");
Console.WriteLine($"  - Microsoft.ML.Probabilistic.Compiler : Compilation Roslyn des modeles");

Infer.NET charge avec succes !
  - Microsoft.ML.Probabilistic.Models : Variables aleatoires et modeles
  - Microsoft.ML.Probabilistic.Distributions : Gaussian, Beta, Dirichlet...
  - Microsoft.ML.Probabilistic.Algorithms : EP, VMP, Gibbs
  - Microsoft.ML.Probabilistic.Compiler : Compilation Roslyn des modeles


### Espaces de noms Infer.NET charges

La cellule precedente a importe les espaces de noms essentiels. Voici leur role :

| Namespace | Contenu | Usage principal |
|-----------|---------|-----------------|
| `Microsoft.ML.Probabilistic.Models` | `Variable<T>`, `VariableArray<T>`, `Range` | Definition des modeles probabilistes |
| `Microsoft.ML.Probabilistic.Distributions` | `Gaussian`, `Beta`, `Gamma`, `Dirichlet` | Distributions de probabilite |
| `Microsoft.ML.Probabilistic.Algorithms` | `ExpectationPropagation`, `VariationalMessagePassing` | Algorithmes d'inference |
| `Microsoft.ML.Probabilistic.Compiler` | `InferenceEngine`, `CompilerChoice` | Compilation et execution |
| `Microsoft.ML.Probabilistic.Math` | Fonctions mathematiques | Operations sur distributions |

> **Architecture Infer.NET** : Le moteur d'inference compile votre modele en code C# optimise, stocke dans `GeneratedSource/`. Cette approche "compile une fois, execute plusieurs fois" offre d'excellentes performances.

## 3bis. Installation de Graphviz (Optionnel)

**Graphviz** permet de visualiser automatiquement les graphes de facteurs generes par Infer.NET.

**Statut actuel** : La cellule precedente a configure le PATH si Graphviz est installe.

**Si Graphviz n'est pas installe** :

```powershell
# Installation via winget (Windows 11)
winget install graphviz

# Puis redemarrez le kernel .NET (Restart dans VSCode)
```

**Alternative sans installation** : Les fichiers `.gv` sont generes et visualisables sur [viz-js.com](https://viz-js.com/)

In [4]:
// Verification de Graphviz (optionnel - le PATH est deja configure)
try
{
    var psi = new System.Diagnostics.ProcessStartInfo {
        FileName = "dot", Arguments = "-V",
        RedirectStandardError = true, RedirectStandardOutput = true,
        UseShellExecute = false, CreateNoWindow = true
    };
    using var proc = System.Diagnostics.Process.Start(psi);
    var stderr = proc.StandardError.ReadToEnd();
    proc.WaitForExit();
    Console.WriteLine((proc.ExitCode == 0 || stderr.Contains("graphviz"))
        ? $"Graphviz OK : {stderr.Trim()}"
        : "Graphviz non fonctionnel");
}
catch 
{ 
    Console.WriteLine("Graphviz non installe - les fichiers .gv seront visualisables sur viz-js.com");
}

Graphviz OK : dot - graphviz version 14.1.2 (20260124.0452)


La verification confirme que Graphviz est operationnel. Les options de visualisation des graphes de facteurs seront disponibles dans les sections suivantes.

---

Passons maintenant a notre premier exemple concret de programmation probabiliste.

## 4. Premier Exemple : Le Lancer de Deux Pieces

### Enonce du probleme

Quelle est la probabilite d'obtenir **deux faces** lors du lancer de deux pieces non biaisees ?

### Solution mathematique

$$P(\text{deux faces}) = P(\text{face}_1) \times P(\text{face}_2) = 0.5 \times 0.5 = 0.25$$

### Solution avec Infer.NET

In [5]:
// Etape 1 : Definition du modele probabiliste
Variable<bool> premierePiece = Variable.Bernoulli(0.5);   // Piece 1 : 50% face
Variable<bool> deuxiemePiece = Variable.Bernoulli(0.5);   // Piece 2 : 50% face
Variable<bool> deuxFaces = premierePiece & deuxiemePiece; // ET logique

// Etape 2 : Creation du moteur d'inference
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;  // Necessaire pour .NET Interactive

// Etape 3 : Execution de l'inference
var resultat = moteur.Infer(deuxFaces);
Console.WriteLine($"Probabilite d'obtenir deux faces : {resultat}");

Compiling model...done.
Probabilite d'obtenir deux faces : Bernoulli(0,25)


### Analyse de la sortie

Le resultat `Bernoulli(0,25)` represente une **distribution de Bernoulli** avec parametre p = 0.25 :

- **Interpretation** : La variable `deuxFaces` a 25% de chance d'etre vraie
- **Coherence mathematique** : 0.5 × 0.5 = 0.25 ✓

Remarquez les messages "Compiling model... done." lors de la premiere execution. Infer.NET **compile dynamiquement** votre modele en code C# optimise, stocke dans un repertoire `GeneratedSource/`. Cette compilation n'a lieu qu'une fois par modele.

> **Pourquoi une distribution et pas juste 0.25 ?**  
> Infer.NET retourne toujours des **distributions completes**, pas seulement des valeurs ponctuelles. Cela permet de propager l'incertitude dans des modeles plus complexes.

## 5. Les 3 Etapes Fondamentales

Tout programme Infer.NET suit ces 3 etapes :

### Etape 1 : Definition du Modele

Creez des **variables aleatoires** et definissez leurs **relations**.

```csharp
// Variables aleatoires avec leurs distributions a priori
Variable<bool> A = Variable.Bernoulli(0.5);
Variable<bool> B = Variable.Bernoulli(0.5);

// Variable derivee (dependante)
Variable<bool> C = A & B;  // C depend de A et B
```

### Etape 2 : Creation du Moteur d'Inference

Le moteur compile le modele et prepare l'algorithme d'inference.

```csharp
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;  // Pour notebooks
```

### Etape 3 : Execution de l'Inference

Demandez au moteur de calculer la distribution d'une variable.

```csharp
var resultat = moteur.Infer(C);  // Distribution marginale de C
```

## 6. Types de Distributions Fondamentales

Infer.NET supporte de nombreuses distributions. Voici les plus courantes :

| Distribution | Type | Usage | Exemple |
|--------------|------|-------|--------|
| **Bernoulli** | Discrete | Evenement binaire | Pile/Face, Vrai/Faux |
| **Discrete** | Discrete | Categorie parmi N | Choix d'un jour, couleur |
| **Gaussian** | Continue | Valeur reelle avec incertitude | Taille, temperature |
| **Gamma** | Continue | Precision, variance | Bruit, fiabilite |
| **Beta** | Continue | Probabilite inconnue | Taux de succes |
| **Dirichlet** | Continue | Vecteur de probabilites | Poids de melange |

In [6]:
// Exemples de creation de variables aleatoires

// Bernoulli : probabilite d'un evenement binaire
Variable<bool> pluie = Variable.Bernoulli(0.3);  // 30% de chance de pluie

// Discrete : choix parmi N options
Variable<int> jourSemaine = Variable.DiscreteUniform(7);  // Uniform sur 0-6

// Gaussian : valeur continue avec incertitude
Variable<double> temperature = Variable.GaussianFromMeanAndVariance(20, 4);  // Moyenne 20, variance 4

// Gamma : precision (inverse de la variance)
Variable<double> precision = Variable.GammaFromShapeAndScale(2, 0.5);

Console.WriteLine("Variables creees avec succes !");

Variables creees avec succes !


### Interpretation des variables creees

La cellule precedente a cree quatre variables aleatoires. Voici ce que represente chacune :

| Variable | Type | Distribution | Parametres | Interpretation |
|----------|------|--------------|------------|----------------|
| `pluie` | `Variable<bool>` | Bernoulli(0.3) | p = 0.3 | 30% de chance de pluie |
| `jourSemaine` | `Variable<int>` | DiscreteUniform(7) | n = 7 | Chaque jour equiprobable (1/7) |
| `temperature` | `Variable<double>` | Gaussian(20, 4) | mu = 20, var = 4 | Temperature moyenne 20 degres, ecart-type 2 |
| `precision` | `Variable<double>` | Gamma(2, 0.5) | shape = 2, scale = 0.5 | Precision avec moyenne 1, mode 0.5 |

**Rappel sur les parametrisations** :

- **Gaussian(mean, variance)** : La variance est le carre de l'ecart-type
- **Gamma(shape, scale)** : Moyenne = shape × scale, mode = (shape - 1) × scale si shape >= 1

> **Attention** : Ces variables ne sont pas encore "liees" entre elles. Dans un modele complet, on definirait des dependances (ex: la temperature influence la probabilite de pluie).

## 7. Exemple Avance : Piece Biaisee

Considerons une piece potentiellement biaisee. Nous observons 7 faces sur 10 lancers. Quel est le biais de la piece ?

### Modele bayesien pour l'estimation de biais

Le code suivant illustre les etapes cles d'un modele bayesien :

1. **Prior** : `Beta(1,1)` = distribution uniforme, aucune croyance initiale sur le biais
2. **Vraisemblance** : Chaque lancer suit `Bernoulli(biais)`
3. **Observations** : 7 faces sur 10 lancers
4. **Inference** : Calcul de la distribution posterieure du biais

> **Beta(1,1)** est un prior "non-informatif" : il donne la meme probabilite a toutes les valeurs de biais entre 0 et 1. C'est equivalent a dire "je n'ai aucune idee a priori du biais de la piece".

In [7]:
// Modele pour estimer le biais d'une piece

// A priori : le biais peut etre n'importe quelle valeur entre 0 et 1
// Beta(1,1) = distribution uniforme sur [0,1]
Variable<double> biais = Variable.Beta(1, 1);

// Observations : 10 lancers
int nombreLancers = 10;
Range lancers = new Range(nombreLancers);
VariableArray<bool> resultats = Variable.Array<bool>(lancers);

// Chaque lancer suit une distribution Bernoulli avec le biais inconnu
using (Variable.ForEach(lancers))
{
    resultats[lancers] = Variable.Bernoulli(biais);
}

// Observations : 7 faces (true), 3 piles (false)
resultats.ObservedValue = new bool[] { true, true, true, true, true, true, true, false, false, false };

// Inference
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Beta biaisPosterieur = moteur.Infer<Beta>(biais);
Console.WriteLine($"Distribution a posteriori du biais : {biaisPosterieur}");
Console.WriteLine($"Moyenne estimee du biais : {biaisPosterieur.GetMean():F3}");
Console.WriteLine($"Intervalle de confiance (ecart-type) : +/- {Math.Sqrt(biaisPosterieur.GetVariance()):F3}");

Compiling model...done.
Distribution a posteriori du biais : Beta(8,4)[mean=0,6667]
Moyenne estimee du biais : 0,667
Intervalle de confiance (ecart-type) : +/- 0,131


### Analyse detaillee du resultat

**Sortie obtenue** : `Beta(8,4)[mean=0,6667]`

La distribution **Beta(α, β)** represente notre croyance mise a jour sur le biais de la piece :

| Parametre | Valeur | Signification |
|-----------|--------|---------------|
| α (alpha) | 8 | 1 (prior) + 7 (faces observees) |
| β (beta) | 4 | 1 (prior) + 3 (piles observes) |
| Moyenne | 0.667 | α / (α + β) = 8/12 |
| Ecart-type | 0.131 | Mesure de l'incertitude residuelle |

**Pourquoi Beta(8,4) et pas simplement 7/10 ?**

1. **Integration du prior** : Nous avons commence avec Beta(1,1), equivalent a avoir deja observe 1 face et 1 pile
2. **Formule de mise a jour** : Beta(α + n_faces, β + n_piles) = Beta(1+7, 1+3) = Beta(8,4)
3. **Moyenne legerement biaisee** : 0.667 vs 0.700 (l'effet du prior diminue avec plus d'observations)

> **Concept cle : Conjugaison**  
> La distribution Beta est **conjuguee** a Bernoulli : prior Beta + observations Bernoulli = posterior Beta.  
> Cette propriete permet une mise a jour analytique exacte, sans approximation.

## 8. Exercice : Trois Pieces

### Enonce

Vous lancez **trois** pieces non biaisees. Calculez :

1. La probabilite d'obtenir **exactement trois faces**
2. La probabilite d'obtenir **au moins deux faces**

### Indice

Pour "au moins deux faces", vous pouvez utiliser plusieurs variables derivees :
- `deuxOuTrois = (p1 & p2) | (p1 & p3) | (p2 & p3)`

### Solution

### Code de l'exercice

La cellule suivante contient la solution complete. Essayez de comprendre chaque ligne avant de l'executer :

- **Lignes 4-6** : Creation de trois variables Bernoulli independantes
- **Ligne 9** : Conjonction logique (AND) des trois variables
- **Ligne 13** : Disjonction logique (OR) de paires - structure plus complexe

In [8]:
// EXERCICE : Completez ce code

// 1. Definir les trois pieces
Variable<bool> piece1 = Variable.Bernoulli(0.5);
Variable<bool> piece2 = Variable.Bernoulli(0.5);
Variable<bool> piece3 = Variable.Bernoulli(0.5);

// 2. Definir "trois faces"
Variable<bool> troisFaces = piece1 & piece2 & piece3;

// 3. Definir "au moins deux faces"
// Hint: (p1&p2) | (p1&p3) | (p2&p3)
Variable<bool> auMoinsDeuxFaces = (piece1 & piece2) | (piece1 & piece3) | (piece2 & piece3);

// 4. Inference
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine($"P(trois faces) = {moteur.Infer(troisFaces)}");
Console.WriteLine($"P(au moins deux faces) = {moteur.Infer(auMoinsDeuxFaces)}");

// Verification mathematique :
// P(trois faces) = 0.5^3 = 0.125
// P(au moins 2) = P(2) + P(3) = C(3,2)*0.5^3 + 0.5^3 = 3*0.125 + 0.125 = 0.5
Console.WriteLine("\nVerification : P(3 faces) = 1/8 = 0.125, P(>=2 faces) = 4/8 = 0.5");

Compiling model...done.
P(trois faces) = Bernoulli(0,125)
P(au moins deux faces) = Bernoulli(0,5781)

Verification : P(3 faces) = 1/8 = 0.125, P(>=2 faces) = 4/8 = 0.5


### Analyse des resultats de l'exercice

**Resultats obtenus** :
- `P(trois faces) = Bernoulli(0,125)` ✓ Exact (1/8)
- `P(au moins deux faces) = Bernoulli(0,5781)` ≠ Valeur theorique (0.5)

**Pourquoi cette difference ?**

La valeur 0.5781 au lieu de 0.5 s'explique par l'**algorithme d'inference approchee** utilise par Infer.NET (Expectation Propagation).

Pour "au moins deux faces", la vraie probabilite est :
- P(exactement 2 faces) = C(3,2) × 0.5³ = 3 × 0.125 = 0.375
- P(exactement 3 faces) = 0.125
- **Total** = 0.5

L'ecart vient de la complexite des **correlations entre variables** dans l'expression `(p1&p2) | (p1&p3) | (p2&p3)`. Les memes variables apparaissent dans plusieurs termes, creant des dependances que l'algorithme EP approxime.

> **Bon a savoir** : Infer.NET utilise des algorithmes d'inference **variationnelle** qui peuvent introduire de legeres erreurs sur des modeles avec des structures de dependance complexes. Pour des calculs exacts sur des modeles discrets simples, d'autres approches (enumeration, junction tree) seraient plus precises.

## 8bis. Troubleshooting et Débogage

Cette section couvre les problèmes courants avec Infer.NET et leurs solutions.

### Erreurs Communes

| Erreur | Cause | Solution |
|--------|-------|----------|
| `Model has no support` | Variable observée incompatible avec le prior | Vérifier que les observations sont dans le support du prior |
| `Improper distribution` | Divergence de l'inférence | Utiliser des priors plus informatifs ou changer d'algorithme |
| `Could not find method` | Opération non supportée | Reformuler le modèle avec des opérations supportées |
| `Compiler error` | Syntax ou type invalide | Vérifier les types des variables et opérateurs |

### Choix de l'Algorithme d'Inférence

| Algorithme | Force | Faiblesse | Usage recommandé |
|------------|-------|-----------|------------------|
| **EP** | Rapide, bon pour gaussiennes | Approximatif, peut diverger | Modèles continus, facteurs mixtes |
| **VMP** | Stable, bon pour discret | Sous-estime l'incertitude | LDA, modèles à composantes |
| **Gibbs** | Exact asymptotiquement | Lent, convergence difficile | Validation, petits modèles |

### Comment changer d'algorithme

```csharp
// Expectation Propagation (défaut)
moteur.Algorithm = new ExpectationPropagation();

// Variational Message Passing
moteur.Algorithm = new VariationalMessagePassing();

// Gibbs Sampling
moteur.Algorithm = new GibbsSampling();
```

### Installation de Graphviz (Optionnel)

Pour visualiser les graphes de facteurs, Infer.NET peut générer des fichiers DOT et les convertir en SVG. Cette fonctionnalité nécessite Graphviz.

**Installation Windows** :

```powershell
# Option 1 : winget (Windows 11)
winget install graphviz

# Option 2 : chocolatey
choco install graphviz

# Option 3 : Téléchargement manuel
# https://graphviz.org/download/
```

**Installation Linux/macOS** :

```bash
# Ubuntu/Debian
sudo apt-get install graphviz

# macOS
brew install graphviz
```

**Vérification** :

```powershell
# Vérifier que dot.exe est dans le PATH
dot -V
```

Si Graphviz n'est pas installé, les fichiers `.gv` (DOT) sont quand même générés et peuvent être visualisés avec des outils en ligne comme [viz-js.com](https://viz-js.com/) ou [edotor.net](https://edotor.net/).

In [ ]:
// Inspection du code genere et du factor graph

Console.WriteLine("=== Options de Debug Infer.NET ===\n");

// Verification de Graphviz (le PATH a ete configure au debut du notebook)
bool graphvizInstalle = false;
try
{
    var psi = new System.Diagnostics.ProcessStartInfo {
        FileName = "dot",
        Arguments = "-V",
        RedirectStandardError = true,
        RedirectStandardOutput = true,
        UseShellExecute = false,
        CreateNoWindow = true
    };
    using var proc = System.Diagnostics.Process.Start(psi);
    var stderr = proc.StandardError.ReadToEnd();
    proc.WaitForExit();
    if (proc.ExitCode == 0 || stderr.Contains("graphviz"))
    {
        Console.WriteLine($"Graphviz disponible : {stderr.Trim()}");
        graphvizInstalle = true;
    }
}
catch (Exception ex) 
{ 
    Console.WriteLine($"Verification Graphviz echouee : {ex.Message}");
}

if (!graphvizInstalle)
{
    Console.WriteLine("Graphviz non detecte - les fichiers .gv seront generes");
    Console.WriteLine("Visualisez-les sur : https://viz-js.com/\n");
}
Console.WriteLine();

// Creation d'un modele simple pour demonstration
Variable<double> x = Variable.GaussianFromMeanAndPrecision(0, 1).Named("x");
Variable<double> y = Variable.GaussianFromMeanAndPrecision(x, 1).Named("y");
y.ObservedValue = 2.0;

// Moteur avec options de debug
InferenceEngine moteurDebug = new InferenceEngine();
moteurDebug.Compiler.CompilerChoice = CompilerChoice.Roslyn;

// Option 1 : Afficher le schedule d'inference
moteurDebug.ShowSchedule = true;
Console.WriteLine("1. ShowSchedule = true : Affiche l'ordre des calculs de messages\n");

// Option 2 : Afficher le factor graph (genere .gv et .svg si Graphviz disponible)
moteurDebug.ShowFactorGraph = true;
Console.WriteLine("2. ShowFactorGraph = true : Genere fichier DOT" + 
    (graphvizInstalle ? " + SVG" : " (SVG si Graphviz disponible)") + "\n");

// Option 3 : Sauvegarder le code genere
moteurDebug.Compiler.WriteSourceFiles = true;
moteurDebug.Compiler.GeneratedSourceFolder = "GeneratedSource";
Console.WriteLine("3. WriteSourceFiles = true : Sauvegarde le code C# genere dans GeneratedSource/\n");

// Option 4 : Montrer les avertissements de compilation
moteurDebug.Compiler.ShowWarnings = true;
Console.WriteLine("4. ShowWarnings = true : Affiche les avertissements du compilateur\n");

// Executer l'inference
Console.WriteLine("--- Execution avec options debug ---\n");
var xPost = moteurDebug.Infer<Gaussian>(x);
Console.WriteLine($"\nResultat : x ~ {xPost}");

// Note : Le FactorGraphHelper est defini plus bas dans ce notebook.
// Une fois charge, utilisez : display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()))
Console.WriteLine("\n--- Factor Graph ---");
Console.WriteLine("Fichiers .gv generes. Voir section FactorGraphHelper ci-dessous pour l'affichage inline.");

### Interpretation du resultat d'inference en mode debug

**Sortie obtenue** : `x ~ Gaussian(1, 0.5)`

Ce resultat represente la **distribution posterieure** de la variable `x` apres avoir observe `y = 2.0`.

| Parametre | Valeur | Signification |
|-----------|--------|---------------|
| Moyenne | 1.0 | Estimation centrale de x |
| Variance | 0.5 | Incertitude sur x (ecart-type ~ 0.71) |

**Pourquoi cette mise a jour ?**

1. **Prior** : x ~ N(0, 1) - nous pensions que x etait proche de 0
2. **Observation** : y = 2.0, avec y ~ N(x, 1)
3. **Posterior** : x ~ N(1, 0.5) - compromis entre le prior et l'observation

Le posterior est "tire" vers l'observation (moyenne passe de 0 a 1) tout en reduisant l'incertitude (variance passe de 1 a 0.5).

> **Formule analytique** (cas gaussien) :  
> La moyenne posterieure = (prior_precision × prior_mean + likelihood_precision × observation) / (prior_precision + likelihood_precision)  
> Soit : (1×0 + 1×2) / (1+1) = 1

### Utilisation des Options de Debug

**Quand utiliser chaque option** :

| Option | Utilité |
|--------|---------|
| `ShowSchedule` | Comprendre l'ordre des messages, identifier les boucles infinies |
| `ShowFactorGraph` | Visualiser le modèle, vérifier les connections |
| `WriteSourceFiles` | Inspecter le code généré, optimiser manuellement |
| `ShowWarnings` | Détecter les approximations, opérateurs expérimentaux |

**Fichiers générés** :

Le dossier `GeneratedSource/` contient :
- `Model_EP.cs` : Code d'inférence pour EP
- `Model_VMP.cs` : Code d'inférence pour VMP
- Ces fichiers sont du C# pur, compilable et exécutable

### Visualisation des Factor Graphs

Lorsque `ShowFactorGraph = true` est activé, Infer.NET génère des fichiers `.gv` (format DOT Graphviz) décrivant la structure du modèle.

**Avec Graphviz installé** :

La conversion en SVG est automatique. Les fichiers générés :
- `Model_*.gv` : Description DOT du graphe de facteurs
- `Model_*.svg` : Visualisation SVG (générée automatiquement)

**Sans Graphviz** :

Les fichiers `.gv` sont quand même générés. Vous pouvez les visualiser :

1. **Outils en ligne** (recommandé pour débuter) :
   - [viz-js.com](https://viz-js.com/) - Coller le contenu du fichier `.gv`
   - [edotor.net](https://edotor.net/) - Éditeur interactif
   - [dreampuf.github.io/GraphvizOnline](https://dreampuf.github.io/GraphvizOnline/) - Visualiseur simple

2. **Conversion locale après installation** :
   ```powershell
   # Installer Graphviz (voir section précédente)
   winget install graphviz
   
   # Convertir un fichier DOT en PNG
   dot -Tpng Model_*.gv -o graph.png
   
   # Ou en SVG
   dot -Tsvg Model_*.gv -o graph.svg
   ```

**Exemple de structure d'un Factor Graph** :

```
[Prior N(0,1)] ----> (Variable x) ----> [GaussianFromMeanAndPrecision] ----> (Variable y = 2.0 observé)
```

Dans ce graphe :
- Les **rectangles** représentent les facteurs (distributions)
- Les **cercles** représentent les variables aléatoires
- Les **flèches** indiquent le flux de messages entre facteurs et variables

### Helper : Affichage inline des Factor Graphs

Le code suivant definit un helper `FactorGraphHelper` pour afficher les graphes de facteurs directement dans les cellules du notebook. Ce helper :
- Detecte automatiquement les fichiers `.gv` generes par Infer.NET
- Les convertit en SVG via Graphviz
- Les affiche inline dans la sortie de la cellule

In [ ]:
// Helper pour afficher les graphes de facteurs (SVG) inline dans le notebook

using Microsoft.DotNet.Interactive.Formatting;

/// <summary>
/// Helper pour generer et afficher les factor graphs Infer.NET dans les notebooks
/// </summary>
public static class FactorGraphHelper
{
    private static bool? _graphvizAvailable = null;
    
    /// <summary>
    /// Verifie si Graphviz (dot) est disponible
    /// </summary>
    public static bool IsGraphvizAvailable()
    {
        if (_graphvizAvailable.HasValue) return _graphvizAvailable.Value;
        try
        {
            var psi = new System.Diagnostics.ProcessStartInfo {
                FileName = "dot", Arguments = "-V",
                RedirectStandardError = true, RedirectStandardOutput = true,
                UseShellExecute = false, CreateNoWindow = true
            };
            using var proc = System.Diagnostics.Process.Start(psi);
            proc.WaitForExit(3000);
            _graphvizAvailable = (proc.ExitCode == 0);
        }
        catch { _graphvizAvailable = false; }
        return _graphvizAvailable.Value;
    }
    
    /// <summary>
    /// Retourne le HTML pour afficher le dernier fichier SVG genere par Infer.NET
    /// Usage: display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()))
    /// </summary>
    public static string GetLatestFactorGraphHtml(int maxWidth = 800)
    {
        // Chercher dans le repertoire courant
        var svgFiles = Directory.GetFiles(Environment.CurrentDirectory, "Model_*.svg");
        
        if (svgFiles.Length > 0)
        {
            var latestSvg = svgFiles
                .OrderByDescending(f => new FileInfo(f).LastWriteTime)
                .First();
            return GetSvgFileHtml(latestSvg, maxWidth);
        }
        else
        {
            // Essayer de convertir le dernier .gv
            return ConvertAndGetLatestGvHtml(maxWidth);
        }
    }

    /// <summary>
    /// Retourne le HTML pour afficher un fichier SVG inline
    /// </summary>
    public static string GetSvgFileHtml(string svgPath, int maxWidth = 800)
    {
        if (!File.Exists(svgPath))
            return $"<div style='color:red'>Fichier non trouve : {svgPath}</div>";

        var svgContent = File.ReadAllText(svgPath);
        var fileName = Path.GetFileName(svgPath);
        return GetSvgContentHtml(svgContent, fileName, maxWidth);
    }
    
    /// <summary>
    /// Retourne le HTML pour afficher du contenu SVG
    /// </summary>
    public static string GetSvgContentHtml(string svgContent, string title = "Factor Graph", int maxWidth = 800)
    {
        return $@"
<div style=""margin: 10px 0; padding: 10px; border: 1px solid #ddd; border-radius: 5px; background: #fafafa;"">
    <div style=""font-weight: bold; margin-bottom: 8px; color: #333;"">{title}</div>
    <div style=""max-width: {maxWidth}px; overflow: auto;"">
        {svgContent}
    </div>
</div>";
    }

    /// <summary>
    /// Convertit un fichier .gv en SVG et retourne le HTML
    /// </summary>
    public static string ConvertAndGetGvHtml(string gvPath, int maxWidth = 800)
    {
        if (!File.Exists(gvPath))
            return $"<div style='color:red'>Fichier .gv non trouve : {gvPath}</div>";

        if (!IsGraphvizAvailable())
        {
            var fileName = Path.GetFileName(gvPath);
            return $@"<div style='padding: 10px; border: 1px solid #f0ad4e; background: #fcf8e3; border-radius: 5px;'>
                <strong>Graphviz non disponible.</strong><br/>
                Copiez le contenu de <code>{fileName}</code> sur <a href='https://viz-js.com/' target='_blank'>viz-js.com</a>
            </div>";
        }

        var svgPath = Path.ChangeExtension(gvPath, ".svg");

        try
        {
            var psi = new System.Diagnostics.ProcessStartInfo
            {
                FileName = "dot",
                Arguments = $"-Tsvg \"{gvPath}\" -o \"{svgPath}\"",
                RedirectStandardError = true,
                UseShellExecute = false,
                CreateNoWindow = true
            };

            using var proc = System.Diagnostics.Process.Start(psi);
            proc.WaitForExit(5000);

            if (proc.ExitCode == 0 && File.Exists(svgPath))
                return GetSvgFileHtml(svgPath, maxWidth);
            else
            {
                var error = proc.StandardError.ReadToEnd();
                return $"<div style='color:red'>Erreur Graphviz : {error}</div>";
            }
        }
        catch (Exception ex)
        {
            return $"<div style='color:red'>Erreur : {ex.Message}</div>";
        }
    }

    /// <summary>
    /// Trouve et convertit le dernier fichier .gv
    /// </summary>
    public static string ConvertAndGetLatestGvHtml(int maxWidth = 800)
    {
        var gvFiles = Directory.GetFiles(Environment.CurrentDirectory, "Model_*.gv");

        if (gvFiles.Length == 0)
        {
            return @"<div style='padding: 10px; border: 1px solid #d9534f; background: #f2dede; border-radius: 5px;'>
                Aucun fichier .gv trouve.<br/>
                Activez <code>ShowFactorGraph = true</code> sur le moteur d'inference.
            </div>";
        }

        var latestGv = gvFiles
            .OrderByDescending(f => new FileInfo(f).LastWriteTime)
            .First();

        return ConvertAndGetGvHtml(latestGv, maxWidth);
    }

    /// <summary>
    /// Nettoie les fichiers .gv et .svg generes
    /// </summary>
    public static int CleanupGeneratedFiles()
    {
        var files = Directory.GetFiles(Environment.CurrentDirectory, "Model_*.gv")
            .Concat(Directory.GetFiles(Environment.CurrentDirectory, "Model_*.svg"));

        int count = 0;
        foreach (var file in files)
        {
            try { File.Delete(file); count++; } catch { }
        }
        return count;
    }
}

Console.WriteLine("FactorGraphHelper charge !");
Console.WriteLine("  - display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml())) : affiche le dernier graphe");
Console.WriteLine("  - FactorGraphHelper.CleanupGeneratedFiles() : nettoie les fichiers generes");

### Utilisation du FactorGraphHelper

Une fois la cellule precedente executee, vous pouvez afficher les graphes de facteurs directement dans le notebook :

```csharp
// Afficher le dernier graphe genere
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

// Nettoyer les fichiers temporaires
FactorGraphHelper.CleanupGeneratedFiles();
```

**Methodes disponibles** :

| Methode | Description |
|---------|-------------|
| `GetLatestFactorGraphHtml()` | Affiche le dernier graphe SVG inline |
| `GetSvgFileHtml(path)` | Affiche un fichier SVG specifique |
| `ConvertAndGetGvHtml(path)` | Convertit un `.gv` en SVG et l'affiche |
| `CleanupGeneratedFiles()` | Supprime les fichiers `.gv` et `.svg` generes |
| `IsGraphvizAvailable()` | Verifie si Graphviz est installe |

## 9. Resume

Dans ce notebook, vous avez appris :

| Concept | Description |
|---------|-------------|
| **Programmation probabiliste** | Representer l'incertitude avec des variables aleatoires |
| **Infer.NET** | Framework Microsoft pour l'inference bayesienne |
| **Les 3 etapes** | Modele -> Moteur -> Inference |
| **Distributions de base** | Bernoulli, Beta, Gaussian, Gamma |
| **Mise a jour bayesienne** | A priori + Observations = A posteriori |

---

## Pour aller plus loin

| Si vous voulez... | Consultez... |
|-------------------|--------------|
| Approfondir les distributions continues | [Infer-2-Gaussian-Mixtures](Infer-2-Gaussian-Mixtures.ipynb) |
| Comprendre les graphes de facteurs | [Infer-3-Factor-Graphs](Infer-3-Factor-Graphs.ipynb) |
| Debugger un modele qui ne converge pas | [Infer-13-Debugging](Infer-13-Debugging.ipynb) |
| Trouver une definition | [Glossaire](Infer-Glossary.md) |

---

## Prochaine etape

Dans le notebook suivant [Infer-2-Gaussian-Mixtures](Infer-2-Gaussian-Mixtures.ipynb), nous approfondirons :

- Les distributions gaussiennes et leur apprentissage
- Les modeles de melange pour les donnees multimodales
- L'utilisation de `Variable.Switch` pour les modeles a composantes

---

## Ressources

- [Documentation Infer.NET](https://dotnet.github.io/infer/)
- [Tutorials officiels](https://dotnet.github.io/infer/userguide/Infer.NET%20tutorials%20and%20examples.html)
- [GitHub dotnet/infer](https://github.com/dotnet/infer)
- [Model-Based Machine Learning Book](https://mbmlbook.com/)

---

## Recapitulatif des distributions utilisees

| Distribution | Notation | Support | Parametres | Utilisation dans ce notebook |
|--------------|----------|---------|------------|------------------------------|
| **Bernoulli** | Bernoulli(p) | {0, 1} | p : probabilite de succes | Lancers de pieces, evenements binaires |
| **Beta** | Beta(a, b) | [0, 1] | a, b : pseudo-comptes | Prior sur le biais de la piece |
| **Gaussian** | N(mu, var) | R | mu : moyenne, var : variance | Temperature, valeurs continues |
| **Gamma** | Gamma(a, s) | R+ | a : forme, s : echelle | Precision, parametres positifs |
| **DiscreteUniform** | Uniform(n) | {0, ..., n-1} | n : nombre de valeurs | Jour de la semaine |

### Concepts probabilistes illustres

| Concept | Description | Exemple |
|---------|-------------|---------|
| **Prior (a priori)** | Croyance avant observation | Beta(1,1) = ignorance totale |
| **Posterior (a posteriori)** | Croyance apres observation | Beta(8,4) apres 7 faces / 3 piles |
| **Conjugaison** | Prior et posterior de meme famille | Beta + Bernoulli = Beta |
| **Inference variationnelle** | Approximation du posterior | EP peut diverger sur modeles complexes |

### Applications pratiques

Les concepts vus dans ce notebook s'appliquent directement a :

- **A/B Testing** : Estimer le taux de conversion avec incertitude
- **Sondages** : Modeliser les marges d'erreur
- **Fiabilite** : Estimer la probabilite de defaillance
- **Diagnostic** : Raisonner sur des symptomes observes